In [ ]:
# COMMAND ----------
# MAGIC
# Import necessary PySpark functions
from pyspark.sql.functions import broadcast

# COMMAND ----------
# MAGIC
# Assuming final_df, scores_df, and aiml_insights_df are already defined DataFrames
# Join the DataFrames using broadcast joins for optimization
comprehensive_df = final_df.join(broadcast(scores_df), "Customer_ID", "inner") \
                           .join(broadcast(aiml_insights_df), "Customer_ID", "inner")

# COMMAND ----------
# MAGIC %md
# The `comprehensive_df` DataFrame now contains the joined data from `final_df`, `scores_df`, and `aiml_insights_df` using the `Customer_ID` as the key.
# The use of `broadcast` helps optimize the join operation by broadcasting the smaller DataFrames across the cluster.
